In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import platform
import seaborn as sns
from sklearn import preprocessing
import re

if platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
    rc('font', family=font_name)

from bs4 import BeautifulSoup
import requests
from datetime import datetime
from datetime import timedelta
import time
from dateutil.relativedelta import relativedelta
from sklearn.linear_model import LinearRegression
from matplotlib import font_manager, rc
font_path = 'C:/Windows/Fonts/NGULIM.TTF'
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

import html
from dateutil import parser

In [5]:
vod_list = pd.read_csv('../final_project/final-project/전처리/vod_data.csv', encoding='cp949')

In [6]:
vod_list = vod_list.drop_duplicates(subset='name')

In [7]:
vod_list = vod_list.reset_index(drop=True)

In [ ]:
def crawling(vod_list):
    for idx, row in vod_list.iterrows():
        # 변수 초기화
        movie_name = ''
        runtime = ''
        daum_director = ''
        director = ''
        summary = ''
        
        a_smry = ''
        a_name = ''
        directors = ''
        # director가 NaN일 때만 채우기
        if pd.isna(row['director']) or row['director'] == '':
            name = row['name']
            bigcategory = row['BigCategory']
            print(name)
            print(bigcategory)
            # 영화 정보 크롤링
            if bigcategory == '영화':
                # 크롤링 검색어준비 - 이름, 러닝타임, 줄거리 - '영화 name' / 감독 - '영화 name 출연진'
                movie_keyword = ' '.join(['영화', name])
                director_keyword = ' '.join([name, '출연진'])
                # url 준비
                movie_url = f"https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={movie_keyword}"
                director_url = f"https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={director_keyword}"
                # 웹페이지 준비
                movie_web_page = requests.get(movie_url).text
                director_web_page = requests.get(director_url).text
                # beautifulsoup 준비
                movie_soup = BeautifulSoup(movie_web_page, 'html.parser')
                director_soup = BeautifulSoup(director_web_page, 'html.parser')
                
                # 파싱 결과 - 영화 이름
                # 못 찾으면 다음에서 검색
                movie_name_html = movie_soup.select('body > div > div > div > div > div > div > div > h2 > span > strong')
                # 네이버에서 못 찾았을 경우 다음에서 검색
                if movie_name_html == []:
                    movie_daum_url = f'https://search.daum.net/search?nil_suggest=btn&w=tot&DA=SBC&q={movie_keyword}'
                    movie_daum_page = requests.get(movie_daum_url).text
                    movie_daum_soup = BeautifulSoup(movie_daum_page, 'html.parser')
                    movie_name_daum_html = movie_daum_soup.select('body > div > div > main > div > div > div > div > div > div > div > div > div > a > b')
                    try: 
                        movie_name = str(movie_name_daum_html).split('>')[1].split('<')[0]
                        print(f'다음 영화 제목 : {html.unescape(movie_name)}')
                    except:
                        print(f'다음 영화 {name} 검색 못함')
                # 네이버에서 찾았을 경우
                else:
                    try:
                        movie_name = str(movie_name_html).split('>')[1].split('<')[0]
                        print(f'네이버 영화 제목 : {html.unescape(movie_name)}')
                    except:
                        print(f'네이버 영화 {name} 검색 못함')
                vod_list.at[idx, 'vod_name'] = html.unescape(movie_name)
                
                # 파싱 결과 - 러닝 타임
                # 러닝 타임 안 나오는 영화는 잘못 검색한 것으로 간주
                runtime_html = movie_soup.select('body > div > div > div > div > div > div > div > div > div > dl > div > dd')
                try:
                    # html parse에서 '분'이 들어간 낱말 가져옴
                    parse_runtime_html = [i for i in str(runtime_html).split('>') if '분' in i][0]
                    runtime = parse_runtime_html.split('<')[0]
                    print(runtime)
                except:
                    # 네이버에 없으면 다음에서 검색
                    movie_runtime_daum_url = f'https://search.daum.net/search?nil_suggest=btn&w=tot&DA=SBC&q={movie_keyword}'
                    movie_runtime_daum_page = requests.get(movie_runtime_daum_url).text
                    movie_runtime_daum_soup = BeautifulSoup(movie_runtime_daum_page, 'html.parser')
                    movie_runtime_daum_html = movie_runtime_daum_soup.select('body > div > div > main > div > div > div > div > div > div > div > div > dl > dd')
                    try:
                        runtime = str(movie_runtime_daum_html[1]).split('<')[-2].split('>')[1].strip()
                        print(runtime)
                        
                        # 다음에서 되면 감독까지는 다음에서 가져옴 - 줄거리는 제공된 데이터 사용
                        daum_director = str(movie_runtime_daum_html[2]).split('</a>')[0].split('>')[-1]
                        vod_list.at[idx, 'director'] = daum_director
                    except:
                        print(f'영화 {name} 러닝타임 없음')
                vod_list.at[idx, 'running_time'] = runtime
                
                time.sleep(5)
                # 파싱 결과 - 감독
                director_html = director_soup.select('body > div > div > div > div > div > div > div > div > div > div > div > div > ul > li')
                try:
                    director = str(director_html).split('<span class="_text">')[1].split('<')[0]
                    # 러닝타임 네이버에 없을 때 다음에서 가져온 감독과 비교 - 대부분 다음이 더 정확해서 다음 감독으로 가져옴
                    if daum_director != '' and director != daum_director:
                        director = daum_director
                    else:
                        print(f'{name} 감독 : {director}')
                except Exception as e:
                    print(e)
                    director_keyword = ' '.join(['영화', html.unescape(movie_name), '출연진'])
                    director_url = f"https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={director_keyword}"
                    director_web_page = requests.get(director_url).text
                    director_soup = BeautifulSoup(director_web_page, 'html.parser')
                    director_html = director_soup.select('body > div > div > div > div > div > div > div > div > div > div > div > div > ul > li')
                    try:
                        director = str(director_html).split('<span class="_text">')[1].split('<')[0]
                        # 러닝타임 네이버에 없을 때 다음에서 가져온 감독과 비교 - 대부분 다음이 더 정확해서 다음 감독으로 가져옴
                        if daum_director != '' and director != daum_director:
                            director = daum_director
                        else:
                            print(f'{name} 감독 : {director}')
                    except Exception as e:
                        print('네이버', e)
                        if daum_director == '':
                            print(f'{name} 감독 못 찾음')
                        else:
                            director = daum_director
                            print(f'다음 {name} 감독 : {daum_director}')
                
                vod_list.at[idx, 'director'] = director
                
                
                # 파싱 결과 - 줄거리
                summary_html = movie_soup.select('body > div > div > div > div > div > div > div > div > div > p')
                if summary_html == []:
                    summary_html = movie_soup.select('body > div > div > div > div > div > div > div > div > div > div > span')
                    if summary_html == []:
                        time.sleep(5)  # 5초 동안 대기
                        movie_keyword = ' '.join([name, '정보'])
                        movie_url = f"https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={movie_keyword}"
                        movie_web_page = requests.get(movie_url).text
                        movie_soup = BeautifulSoup(movie_web_page, 'html.parser')
                        summary_html = movie_soup.select('body > div > div > div > div > div > div > div > div > div > p')
                        if summary_html == []:
                            summary_html = movie_soup.select('body > div > div > div > div > div > div > div > div > div > div > span')
                            if summary_html == []:
                                print(f'영화 {name} 줄거리 못 찾음')
                                time.sleep(5)
                            else:
                                summary = str(summary_html[0]).split(">")[1].split("<")[0]
                                vod_list.at[idx, 'Summary'] = html.unescape(summary)
                                print(html.unescape(summary))
                        else:
                            summary = str(summary_html[0]).split(">")[1].split("<")[0]
                            vod_list.at[idx, 'Summary'] = html.unescape(summary)
                            print(html.unescape(summary))
                    else:
                        summary = str(summary_html[0]).split(">")[1].split("<")[0]
                        vod_list.at[idx, 'Summary'] = html.unescape(summary)
                        print(html.unescape(summary))
                else:
                    summary = str(summary_html[0]).split(">")[1].split("<")[0]
                    vod_list.at[idx, 'Summary'] = html.unescape(summary)
                    print(html.unescape(summary))
            
                time.sleep(5)
            
            # 영화 외 정보 가져오기 - 이름, 줄거리, 연출자
            else:
                # 줄거리
                asset_name = ' '.join([name, '소개'])
                asset_url = f"https://search.daum.net/search?nil_suggest=btn&w=tot&DA=SBC&q={asset_name}"
                asset_web_page = requests.get(asset_url).text
                a_soup = BeautifulSoup(asset_web_page, 'html.parser')
                smry_list = a_soup.select('body > div > div > main > div > div > div > div > div > div > div > div > div > div > dl > dd')
                if smry_list == []:
                    print()
                else:
                    try:
                        a_smry = str(smry_list).split('class="highlighting">')[1].split('<')[0]
                        print(html.unescape(a_smry))
                    except:
                        try:
                            a_smry = str(smry_list[0]).split(">")[1].split("<")[0]
                            print(html.unescape(a_smry))
                        except:
                            print()
                vod_list.at[idx, 'Summary'] = html.unescape(a_smry)
                
                # 이름
                name_list = a_soup.select('body > div > div > main > div > div > div > div > div > div > div > span > a')
                try:
                    a_name = str(name_list).split('<')[-2].split('>')[-1]
                    print(f'{name} 이름 : {html.unescape(a_name)}')
                except:
                    print(f'{name} 이름 못 찾음')
                vod_list.at[idx, 'vod_name'] = html.unescape(a_name)
                
                # 연출자
                asset_name = ' '.join([name, '제작'])
                asset_url = f"https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&sug=&sugo=&sq=&o=&q={asset_name}"
                asset_web_page = requests.get(asset_url).text
                a_soup = BeautifulSoup(asset_web_page, 'html.parser')
                a_list = a_soup.select('body > div > div > main > div > div > div > div > div > div > div > div > div > div > dl > dd')
                if a_list == []:
                    a_list = a_soup.select('body > div > div > main > div > div > div > div > div > div > div > div > div > div > dl > dd > span > span')
                try:
                    infos = str(a_list[1]).split('<span class="highlighting">')[1:]
                    director_list = [info.split('<')[0] for info in infos]
                    directors = ", ".join(director_list)
                    print(directors)
                except:
                    print(f'{name} 연출자 못 찾음')
                vod_list.at[idx, 'director'] = directors

In [49]:
vod = pd.read_csv('vod_list_addinfo.csv', encoding='cp949')

In [50]:
crawling(vod)

핑크퐁 동물 동요
키즈

핑크퐁 동물 동요 이름 못 찾음
핑크퐁 동물 동요 연출자 못 찾음
라바 키즈동요 Part4
키즈

라바 키즈동요 Part4 이름 못 찾음
라바 키즈동요 Part4 연출자 못 찾음
지니강이 플러스 시즌7
키즈
친구들 안녕! 지니와 강이와 함께 신나는 장난감 놀이와 두근두근 대결놀이까지! 함께 신나게 놀아요~! 어린이 프로그램
지니강이 플러스 시즌7 이름 : 지니강이 플러스 시즌7
지니강이 플러스 시즌7 연출자 못 찾음
엉뚱남매 요리조리쇼
키즈

엉뚱남매 요리조리쇼 이름 못 찾음
엉뚱남매 요리조리쇼 연출자 못 찾음
라바 키즈동요 Part3
키즈

라바 키즈동요 Part3 이름 못 찾음
라바 키즈동요 Part3 연출자 못 찾음
지니강이 플러스 시즌9
키즈

지니강이 플러스 시즌9 이름 못 찾음
지니강이 플러스 시즌9 연출자 못 찾음
엘리가간다 시즌12
키즈

엘리가간다 시즌12 이름 못 찾음
엘리가간다 시즌12 연출자 못 찾음
지니강이 플러스 시즌8
키즈
친구들 안녕! 지니와 강이와 함께 신나는 장난감 놀이와 두근두근 대결놀이까지! 함께 신나게 놀아요~! 어린이 프로그램
지니강이 플러스 시즌8 이름 : 지니강이 플러스 시즌8
지니강이 플러스 시즌8 연출자 못 찾음
헤이지니 럭키강이 Part1
키즈

헤이지니 럭키강이 Part1 이름 못 찾음
헤이지니 럭키강이 Part1 연출자 못 찾음
우당탕탕 에디의 실험실
키즈

우당탕탕 에디의 실험실 이름 못 찾음
우당탕탕 에디의 실험실 연출자 못 찾음
티티체리
키즈
색다른 장소에서 펼쳐지는 방탈출 게임과 소녀들의 신나는 액션 모험! 매번 새로워지는 방탈출 게임은 아이들도 주도적으로 문제를 풀어보도록 자연스럽게 이끌며, 두뇌 활동을 할 수 있도록 독려하는 애니메이션
티티체리 이름 : 티티 체리
티티체리 연출자 못 찾음
달려라 캐리
키즈

달려라 캐리 이름 못 찾음
달려라 캐리 연출자 못 찾음
핑크퐁 3D 율동 동요
키즈

핑크퐁 3D 율동 동요 이름 못 찾음
핑크퐁 3D 율동 동요 연출자 못 찾음
타요의

아내가결혼했다
영화
네이버 영화 제목 : 아내가 결혼했다
119분
아내가결혼했다 감독 : 정윤수
귀여운 외모와 넘치는 애교, 헌책을 사랑하는 지적인 면모와 남자 못지 않은 축구에 대한 지식과 열정을 가지고 있는 인아. 말까지 척척 잘 통하는 그녀를 만날수록 덕훈은 보통 여자와 다른 그녀의 특별한 매력에 점점 빠져든다. 그러나 평생 그녀만을 사랑하고픈 덕훈과는 달리, 덕훈을 사랑하지만 그’만’을 사랑하는 것은 아니며, 사랑하는 사람’들’과 사랑하며 살고 싶다는 너무나 자유로운 그녀. “하지만 지금 중요한 건 그녀가 나를 사랑한다는 것! 나를 사랑하는 한 그녀는 내꺼라는 것!” 그녀의 핸드폰이 꺼져있던 어느 날, 불안함에 폭발하여 따져 묻는 덕훈에게 인아는 다른 남자와 잤다는 충격적인 고백을 한다. 홧김에 이별을 선언하지만, 잊으려 해도 잊을 수 없고, 커져만 가는 그녀에 대한 마음에 괴로운 덕훈. 오랜 고민 끝에 내린 결론. 평생 한 사람만 사랑할 자신이 없다는 그녀를 독점할 수 있는 유일한 방법은 결혼뿐이다. “너의 자유로운 연애를 종식시키기 위해, 너를 연애의 무덤 ‘결혼’으로 데려가리라!” 결국 그녀의 자유로운 연애를 받아들이는 조건으로 결혼에 성공하는 덕훈. 매일 밤 축구를 관람하며 즐기는 섹스와 완벽한 요리 솜씨는 덕훈을 최고로 행복하게 만든다. 하지만 또 한번의 충격 고백. 사랑하는 남자가 생겼다는 인아는 그 놈과도 결혼을 하겠다는 상상도 못할 제안을 한다. 과연 그 놈을 무찌를 것인가? 그녀를 포기할 것인가? 아니면.. 그녀의 반만이라도 가질 것인가?!
레이디
영화
네이버 영화 제목 : 레이디
115분
list index out of range
레이디 감독 : 오성수
누구나 올 수 있는 공간, 그러나 그 사람이 누구인지 알 수 없는 공간인 게스트하우스에 각자의 이유를 가진 사람들이 모여든다. 비밀을 품고 있는 여주인, 여정이가 운영하는 게스트하우스 ‘파란 나라’에 어느 날 어두운 기운을 내뿜는 송일도가 찾아오고 파란나라에서는 예상치 못했던 사건이 발

우리 사랑이 향기로 남을 때
영화
네이버 영화 제목 : 우리 사랑이 향기로 남을 때
108분
우리 사랑이 향기로 남을 때 감독 : 임성용
삶에 치여 제대로 된 연애 한 번 못해본 남자 ‘창수’(윤시윤) 낯선 이에게 받은 향수를 뿌리자마자 여자들이 달려든다?! 가족에 치여 누굴 좋아해본 적도 없는 것 같은 여자 ‘아라’(설인아) 어느 날, 매일같이 타던 버스에서 나는 향기에 두근대기 시작한다 ‘창수’에게 이끌린 ‘아라’는 영문도 모른 채 사랑에 빠지고, 서툴러도 조금씩 사랑을 키워나가던 그때! 갑작스럽게 등장한 전 애인 ‘제임스’(노상현)가 폭로한 ‘창수’의 비밀! 내가 사랑에 빠진 게, 향수 때문이라고? 2023년 2월, 마법 같은 로맨스가 시작된다!
몽정기: 여름날의 추억
영화
네이버 영화 제목 : 몽정기: 여름날의 추억
93분
몽정기: 여름날의 추억 감독 : 손시
안후이의 한 작은 마을, 한창 이성에 관심이 많은 14살 소년 사오제와 그의 단짝 친구 쿠과는 비디오 대여점에서 야한 영화를 빌려 부모님 몰래 집에서 보곤 한다 그런던 중 두 소년은 비디오가게에서 우연히 만난 소녀 잉쯔에게 호감을 느끼고 쿠과는 사오제에게 야한 영화를 빌려서 잉쯔와 함께 보자고 제안한다. 한편 동네 선배의 방해로 두 친구는 곤경에 빠지고, 그 선배로부터 구타를 당한다. 화가 난 두 친구는 선배를 골탕먹이기 위해 작전을 세우고 잉쯔와 다시 비디오를 같이 보기로 약속하는데...
내 아내 이야기
영화
네이버 영화 제목 : 내 아내 이야기
169분
내 아내 이야기 감독 : 일디코 엔예디
1년 365일을 바다 위에서 지내는 선장 ‘야코프’는 매일 반복되는 일상에 지쳐 있다. 어느 날, 친구에게 농담 삼아 던진 말을 시작으로 그의 인생은 특별한 2막을 맞이하게 되는데… “지금 카페에 들어오는 손님과 결혼할게” “그녀는 내 평생의 사랑이 될 테니까”
두여자의집
영화
네이버 영화 제목 : 두 여자의 집
123분
list index out of range
두여자의집 감독 : 곽지균
상실된 옛사랑으

무지개 여신
영화
네이버 영화 제목 : 무지개 여신
104분
무지개 여신 감독 : 이문영
첼리스트 지석이 변심한 애인의 결혼식을 망쳐놓고 도망치 듯 외딴 어촌 마을로 흘러든다. 낚시 출조 관련 장사를 하는 지연의 도움으로 하룻밤을 보낸 지석은 ‘김양’이라 불리는 이 여인이 궁금하다. 그런데 이 마을 할아버지들이 수상하다. 볕 좋은 양지에 앉아 해바라기를 하며 지연의 뽀얀 속살을 훔쳐보는 할아버지들....... 할아버지들이 지연을 유혹하려는 지석을 위협한다. 그러나 지석에게 깡촌 할아버지들은 상대할 가치도 없는 비천한 존재일 뿐이다. 할아버지들의 경고를 무시하고 지석은 지연과 함께 떠나기로 한다.
후궁제왕의첩
영화
네이버 영화 제목 : 후궁 : 제왕의 첩
122분
후궁제왕의첩 감독 : 김대승
살기 위해 변해야 했던 여인 ‘화연’ 권력과 사랑으로 벌거벗은 왕 ‘성원대군’ 모든 걸 빼앗긴 남자 ‘권유’ 죽기 전엔 나갈 수 없는 지독한 궁에서 일어난 미친 욕망
사랑과 거짓말 
영화
네이버 영화 제목 : 사랑과 거짓말
107분
list index out of range
사랑과 거짓말  감독 : 후루사와 켄
정부가 결혼 상대를 정해주는 연애 금지의 세계. 만 16살 생일에 거짓말같이 완벽한 남자 ‘소스케’가 나에게 다가왔다. ‘좋아하게 된 것 같아’ 오랜 시간 내 곁을 지켜 온 소꿉친구 ‘시바’. 이제 다시는 그를 볼 수 없다고 한다. 거.짓.말! ‘왠지 아쉬워’ 서툰 마음이 새어 나가는 지금. 나, 어디로 향하는 걸까
가베:그녀의숲
영화
네이버 영화 제목 : 가베: 그녀의 숲
101분
가베:그녀의숲 감독 : 마르쿠스 카스트로
전쟁으로 인해 고아가 된 ‘모니르’는 사촌 형을 따라 스웨덴으로 망명한다. 난민 수용 시설 부족으로 머물게 된 어느 시골 마을, 마을 축제에서 우연히 마주친 소녀에게 ‘모니르’는 강한 끌림을 느낀다. 난민을 멸시하는 마을 주민들을 피해 찾아간 숲, ‘모니르’는 그때 그 소녀 ‘모아’를 다시 만난다. 전쟁의 트라우마로 괴로웠던 소년은 마침내 소녀의 

웨이트: 감각에 눈 뜰때
영화
네이버 영화 제목 : 웨이트: 감각에 눈뜰 때
97분
list index out of range
웨이트: 감각에 눈 뜰때 감독 : 아쿠 로히미스
감각에 눈을 뜨고 자기 위로를 하는 그녀. 목사 남편은 친구를 부부의 섬으로 초대하고 그는 목사와 아내의 관계를 몰래 훔쳐본다. 시선을 느끼는 그녀는 욕망과 유혹에 갈등하지만 위험해서 더 달콤한 불륜의 경계선을 서서히 넘는데… 마침내 절정을 만나다!
감동주의보
영화
네이버 영화 제목 : 감동주의보
98분
감동주의보 감독 : 김우석
감동받으면 죽는다고?! 희귀질환인 감동병을 앓는 컬링 천재 보영(홍수아). 평생을 함께하고 싶은 시골청년 ‘철기’(최웅)를 만나 인생 최고의 순간들을 맞이한다. 어느날, 코피가 멈추게 하는 방법(?)을 찾게 되면서 포기했던 컬링을 다시 도전하게 되는데… 보영은 감동병을 이기고 꿈과 사랑 두 마리 토끼를 다 잡을 수 있을까?
류선비의혼례식
영화
네이버 영화 제목 : 류선비의 혼례식
96분
list index out of range
류선비의혼례식 감독 : 박건호
집안끼리 정략으로 이뤄진 혼례식에 신부가 남자였다? 사라진 누이 대신 혼례를 치룬 '최기완'과 얼떨결에 여장 남자를 아내로 맞게 된 '류호선'. 이들의 파란만장한 결혼 생활을 다룬 조선막장사기로맨스! 상대의 얼굴부터 확인하는 지금과는 달리 조선 시대에는 평생을 같이 살아야 할 자신의 반려자와 얼굴도 보지 못한 채 결혼식을 올리는 일이 훨씬 많았다. 그로 인해 상대가 바뀌어도 그 사실을 알 수 없었는데, 심지어 그 상대가 남자라니! 좌충우돌 신혼생활의 코미디, 그 안에서 생긴 서로에 대한 애틋함을 그린다.
오 루시!
영화
네이버 영화 제목 : 오 루시!
95분
오 루시! 감독 : 히라야나기 아츠코
친구도, 가족도, 사랑도 없는 외로운 중년 여성 세츠코. 어느 날, 엉뚱한 조카 미카의 권유로 영어 학원을 등록하게 되고 그곳에서 꽃미남 영어강사 '존'에게 첫눈에 반한다. 금발의 가발을 쓰고 '루시‘라는 새 이름을

...

시체가 돌아왔다
영화
네이버 영화 제목 : 시체가 돌아왔다
110분
시체가 돌아왔다 감독 : 우선호
매사 이성적이고 치밀한 연구원 ‘현철’과 매번 말보다 행동이 앞서는 ‘동화’ 공통점 없는 이들은 오로지 시체를 훔쳐 각자의 목적을 이루기 위해 의기투합한다. 하지만 모든 게 완벽해 보였던 그들의 계획은 예상치 못한 인물 ‘진오’의 등장으로 위기를 맞게 되고, 목적은 다르지만, 원하는 단 하나! 시체를 훔치기 위해 셋은 한 팀이 되어 본격적인 작전에 나선다. 하지만 그와 함께 또 다른 목적으로 시체, 그리고 이들을 쫓는 일행들의 추격이 시작되는데…! 치밀한 브레인의 능력자 ‘현철’, 뼛속까지 다크한 행동파 ‘동화’, 그리고 천부적 사기본능의 ‘진오’이제 그들의 세상에 없던 대담한 플레이가 시작된다!
7호실
영화
네이버 영화 제목 : 7호실
100분
7호실 감독 : 이용승
서울의 망해가는 DVD방 사장 두식(신하균). 학자금 빚을 갚으려 DVD방에서 일하는 알바생 태정(도경수). 팔리지도 않던 가게에 기적처럼 매수자가 나타난 바로 그 때! 예상치 못한 사고가 일어나고, 두식은 시체를 7호실에 숨겨 봉쇄한다. 한편, 빚을 해결해주는 조건으로 마약을 7호실에 잠시 감춰놨던 태정은 늘 열려있던 그 방의 문을 두식이 갑자기 잠가버리자 당황하는데… 닫아야 사는 사장 vs 열어야 사는 알바생! 두 남자의 생존이 걸린 문제의 방 ‘7호실’
염력
영화
네이버 영화 제목 : 염력
101분
list index out of range
염력 감독 : 연상호
평범한 은행 경비원 ‘석헌’(류승룡). 어느 날 갑자기 그의 몸에 이상한 변화가 찾아온다. 생각만으로 물건을 움직이는 놀라운 능력, 바로 염력이 생긴 것. 한편, ‘민사장’(김민재)과 ‘홍상무’(정유미)에 의해 ‘석헌’의 딸, 청년 사장 ‘루미’(심은경)와 이웃들이 위기에 처하게 되고... ‘석헌’과 ‘루미’, 그리고 변호사 ‘정현’(박정민)이 그들에 맞서며 놀라운 일이 펼쳐지는데...! 어제까진 초평범, 하루아침에 초능력 이제 그의

의처소동
영화
네이버 영화 제목 : 의처 소동
87분
의처소동 감독 : 심우섭
아내를 사랑하는 말단사원 삼용은 집마련을 위해 아내 명임의 취직을 허락한다. 그러나 뜻밖에 명임은 사장의 비서로 들어오게 되고 남자 사원들은 명임에게 반한다. 불안한 삼용은 아내에게 사표를 권했으나 집마련을 위해 좀더 참기로 한다. 그러던 어느날 동기가 명임과 동행하느라 집으로 들어갈 수 없게 된 삼용은 만취해서 정신을 잃고 어느 여인과 호텔에서 밤을 지낸다. 명임은 그 바람에 사장의 부탁을 들어주기로 한다. 삼용은 극도의 노이로제 증세로 명임과 서사장을 의심한다. 그러나 서사장은 명임과 삼용과의 사이를 알고 난 후 모든 것을 이해하고 명임은 사표를 낸다. 그리고 삼용은 노이로제 증세의 치료를 위해 정신병원으로 간다.
나 홀로 집에 2뉴욕을 헤메다
영화
네이버 영화 제목 : 나 홀로 집에 2 - 뉴욕을 헤매다
115분
나 홀로 집에 2뉴욕을 헤메다 감독 : 크리스 콜럼버스
캐빈은 가족과 함께 플로리다로 크리스마스 휴가를 떠나기로 한다. 그러나 공항의 소란속에서 그만 뉴욕행 비행기에 오르게 되어 거대한 도시에 혼자 남게된다. 홀로 뉴욕에서 해방감을 느끼는 캐빈은 아버지의 카드로 플로리다 호텔에 투숙하는 한편 덩컨 장난감가게에서 만난 덩컨씨가 크리스마스때 번 돈을 어린이병원에 기부하려는 계획을 알게된다. 그러나 이때 캐빈은 감옥에서 탈출한 마브와 해리를 덩컨가게 앞에서 만나 쫓기다가 해리와 마브에게 잡히고 만다. 여기에서 해리와 마브가 이 기부금을 노린다는 사실을 알게된 케빈은 간신히 도망쳐 삼촌의 집을 이용해 이 두 도둑을 저지시킬 준비에 들어간다. 결국 캐빈에게 혼이난 두 멍청한 도둑들은 공원에서 캐빈의 신고로 출동한 경찰에 잡히고, 캐빈은 카드를 추적해 찾아온 엄마와 극적인 상봉을 한다.
탐정: 리턴즈
영화
네이버 영화 제목 : 탐정: 리턴즈
116분
탐정: 리턴즈 감독 : 이언희
역대급 미제 사건을 해결한 추리 콤비, 셜록 덕후 만화방 주인 ‘강대만’(권상우)과 광역수사대 레전드


문장 모험 이름 못 찾음
문장 모험 연출자 못 찾음
오싹오싹 이야기 시즌2
키즈

오싹오싹 이야기 시즌2 이름 못 찾음
오싹오싹 이야기 시즌2 연출자 못 찾음
임신부요가
키즈

임신부요가 이름 못 찾음
임신부요가 연출자 못 찾음
핑크퐁 공룡 동요
키즈

핑크퐁 공룡 동요 이름 못 찾음
핑크퐁 공룡 동요 연출자 못 찾음
핑크퐁 버스송
키즈

핑크퐁 버스송 이름 못 찾음
핑크퐁 버스송 연출자 못 찾음
뽀로로와 노래해요 NEW 1
키즈
아이들의 상상력을 자극하는 재미난 내용으로 된 가사에 신나는 율동을 곁들여 뽀로로 친구들과 함께 아이들이 쉽게 따라 부르고 동작을 익히며 가벼운 운동도 할 수 있는 프로그램
뽀로로와 노래해요 NEW 1 이름 : 뽀로로와 노래해요 NEW1
뽀로로와 노래해요 NEW 1 연출자 못 찾음
브레드이발소 시즌3 Part1
키즈

브레드이발소 시즌3 Part1 이름 못 찾음
브레드이발소 시즌3 Part1 연출자 못 찾음
포켓몬스터: 리코와 로드의 모험
키즈
꿈과 비밀로 가득 찬 포켓몬스터 세계에 설렘과 용기를 안고 발걸음을 내디딘 새로운 두 명의 주인공, 신비한 펜던트를 가진 팔데아 출신 소녀 '리코' 수수께끼의 몬스터볼을 가진 관동 출신 소년 '로드'의 새로운 모험이야기
포켓몬스터: 리코와 로드의 모험 이름 : 포켓몬스터: 리코와 로드의 모험
덴사오리
매직팬던트 대모험
키즈
말로 설명하기 힘든 부모님에 대한 효, 사람간의 정, 가족간의 사랑 등 우리나라에만 있는 독특하고 매력적인 문화를 소개하는 애니메이션
매직팬던트 대모험 이름 : 매직펜던트 대모험
매직팬던트 대모험 연출자 못 찾음
닌자고 드래곤 헌터
키즈
가마돈이 닌자고 시티를 점령하자 낙담하던 로이드가 드래곤 전함의 잔해를 발견하고 과거로부터 돌아온 원소 마스터들의 도움을 받게 되는 이야기
닌자고 드래곤 헌터 이름 : 닌자고 드래곤 헌터
닌자고 드래곤 헌터 연출자 못 찾음
프렌즈 2022
키즈
새로운 모습으로 찾아온 프렌즈 친구들~ 즐거움이 가득한 하트레이크 시티로 떠나보자! 언제나 함께

형사 Duelist
영화
네이버 영화 제목 : 형사 Duelist
111분
형사 Duelist 감독 : 이명세
어지러운 세상, 형사 '남순'(하지원)과 파트너 안포교(안성기)는 가짜 돈이 유통되자 범인 색출을 위해 필사의 노력을 한다. 사건의 진실에 다가갈수록 '슬픈눈'(강동원)이라 불리는 자객이 수면위로 드러난다. '남순'과 '슬픈눈'은 숙명적인 대결은 시작되었다. 쫓는 자와 쫓기는 자의 돌이킬 수 없는 대결. 맞붙은 순간, 낯선 감정이 두 사람을 혼란스럽게 한다. 서로를 바라보는 '남순'과 '슬푼눈'의 눈빛이 흔들린다. 하지만 갈 길이 다른 두 사람, 그들에게는 대결도 사랑인 듯하다. 상대를 죽이지 않으면 내가 죽을 수 밖에 없는 숙명적인 갈등에서 그들의 사랑은 더욱 애절하다.
알라딘
영화
네이버 영화 제목 : 알라딘
128분
list index out of range
알라딘 감독 : 가이 리치
머나먼 사막 속 신비의 아그라바 왕국의 시대. 좀도둑 ‘알라딘’은 마법사 ‘자파’의 의뢰로 마법 램프를 찾아 나섰다가 주인에게 세 가지 소원을 들어주는 지니를 만나게 되고, 자스민 공주의 마음을 얻으려다 생각도 못했던 모험에 휘말리게 되는데…
더 독: 무법도시
영화
네이버 영화 제목 : 더 독: 무법도시
125분
더 독: 무법도시 감독 : 브와디스와프 파시코브스키
거대 조직의 간부를 허무하게 놓친 경찰 ‘비트코프스키’. 이제는 함께 일한 동료들마저 의심스럽다. 한편 무기징역 선고 후 25년 만에 출소한 ‘마우레르’는 사라진 아들을 찾아달라는 옛 파트너 ‘모라비에츠’의 부탁을 받아 조사에 나선다. 각자의 사건을 쫓는 중 마주한 세 사람은 이 모든 것이 하나로 연결되고 있다는 사실을 알게 된다. 도시 속에 숨은 악을 처리하기 위해 비공식 협조를 시작한 경찰과 범죄자. 이제 법의 테두리를 벗어난 진짜 심판이 시작된다!
로스트리전로마군단의 반란
영화
네이버 영화 제목 : 로스트리전: 로마군단의 반란
107분
로스트리전로마군단의 반란 감독 : 다비드 코카
황제인 네피스의 존

꽃에서 나온 코끼리 연출자 못 찾음
수박 수영장
키즈

수박 수영장 이름 못 찾음
수박 수영장 연출자 못 찾음
한글탐정둘리
키즈

한글탐정둘리 이름 못 찾음
한글탐정둘리 연출자 못 찾음
장수탕 선녀님
키즈

장수탕 선녀님 이름 못 찾음
장수탕 선녀님 연출자 못 찾음
Who stole the Cookies from the Cookie Jar?
키즈

Who stole the Cookies from the Cookie Jar? 이름 못 찾음
Who stole the Cookies from the Cookie Jar? 연출자 못 찾음
이파라파냐무냐무
키즈

이파라파냐무냐무 이름 못 찾음
이파라파냐무냐무 연출자 못 찾음
선협 로맨스: 옥소령 
TV드라마
이단 세력을 처단하기 위해 인간 세상에 내려온 봉래의 상선, 단목취. 계봉부의 미남 포졸 전안이 만나서 범죄를 해결하고 사랑을 찾아가는 이야기를 그린 드라마
선협 로맨스: 옥소령  이름 : 옥소령
정위문, 원수위
교약운간월
TV드라마
운천월은 용경 세자와 이리저리 얽히다 결국 사랑에 빠지지만, 운천월은 용경 세자를 치료하기 위해 애쓰다 기억을 잃게 되고, 두 사람은 갖은 고생 끝에 돌고 돌아 끝내 함께하게 되는 이야기
교약운간월 이름 : 교약운간월
곽호
일편빙심재옥호 : 강호의 수사관
TV드라마
천재소녀 막염은 오사형의 누명을 벗기고자 강호에 발을 들이고, 개봉부까지 입성한다. 츤데레 전조와 미운 정 고운 정이 들며 '어묘신견' 커플이 되어 켜켜이 쌓인 음모를 풀어내는 이야기
일편빙심재옥호 : 강호의 수사관 이름 : 일편빙심재옥호 : 강호의 수사관
조금도
이과생추입정망
TV드라마
남자로 가득한 공대지만 쑤청청은 솔로를 탈출하지 못한다. 그러던 어느 날, 쑤청청은 연구소 우등생 샤오위허와 기괴하면서 낭만적인 만남을 갖게 되는 이야기를 그린 드라마
이과생추입정망 이름 : 이과생추입정망
이과생추입정망 연출자 못 찾음
하선생적연연불망
TV드라마
교통사고로 말문을 닫은 아들과 까칠하고 차가운 아빠가 따뜻한 마음을 가진 아동심리

In [52]:
column_names = vod.columns
column_names
vod = vod.drop([i for i in column_names if 'Unnamed' in i], axis=1)

In [53]:
vod

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY,vod_name,running_time
0,핑크퐁 동물 동요,,핑크퐁,,키즈,기타,키즈,0:01,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",,NaN
1,라바 키즈동요 Part4,,"엘로우,레드",,키즈,기타,키즈,0:02,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...,,NaN
2,지니강이 플러스 시즌7,친구들 안녕! 지니와 강이와 함께 신나는 장난감 놀이와 두근두근 대결놀이까지! 함께...,"헤이지니,럭키강이",,키즈,기타,키즈,0:11,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...,지니강이 플러스 시즌7,NaN
3,엉뚱남매 요리조리쇼,,"기원,예현,홍윤화",,키즈,기타,키즈,0:11,먹을 수 있는 물병. 지구 환경을 지키자는 엉뚱남매. 하지만 말처럼 쉽게 지켜지지가...,,NaN
4,라바 키즈동요 Part3,,-,,키즈,기타,키즈,0:02,CLEAN UP SONG. Clean up Clean up everybody cle...,,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4711,엄마가 화났다,,-,,키즈,학습,키즈,0:08,인기작가 최숙희의 그림책. 아이들에게 건네는 화해와 위로의 메시지! 아이와 엄마 사...,,NaN
4712,핑크퐁 자동차 동화,,핑크퐁,,키즈,학습,키즈,0:04,Lil Reds First Fire(용감한 소방차 뚜뚜와 뿌뿌). 불이 난 곳이면 ...,,NaN
4713,간질간질,,-,,키즈,학습,키즈,0:06,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,,NaN
4714,월간 아기상어,,아기상어,,키즈,학습,키즈,0:01,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,,NaN


In [54]:
vod.loc[vod['Summary'] == '', 'Summary'] = vod['SMRY']

In [56]:
vod.loc[vod['running_time'] == '', 'running_time'] = vod['runningtime']

In [58]:
vod['running_time'] = vod['running_time'].fillna(vod['runningtime'])

In [60]:
vod.loc[vod['vod_name'] == '', 'vod_name'] = vod['name']

In [61]:
vod

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY,vod_name,running_time
0,핑크퐁 동물 동요,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,,키즈,기타,키즈,0:01,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁 동물 동요,0:01
1,라바 키즈동요 Part4,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...,"엘로우,레드",,키즈,기타,키즈,0:02,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...,라바 키즈동요 Part4,0:02
2,지니강이 플러스 시즌7,친구들 안녕! 지니와 강이와 함께 신나는 장난감 놀이와 두근두근 대결놀이까지! 함께...,"헤이지니,럭키강이",,키즈,기타,키즈,0:11,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...,지니강이 플러스 시즌7,0:11
3,엉뚱남매 요리조리쇼,먹을 수 있는 물병. 지구 환경을 지키자는 엉뚱남매. 하지만 말처럼 쉽게 지켜지지가...,"기원,예현,홍윤화",,키즈,기타,키즈,0:11,먹을 수 있는 물병. 지구 환경을 지키자는 엉뚱남매. 하지만 말처럼 쉽게 지켜지지가...,엉뚱남매 요리조리쇼,0:11
4,라바 키즈동요 Part3,CLEAN UP SONG. Clean up Clean up everybody cle...,-,,키즈,기타,키즈,0:02,CLEAN UP SONG. Clean up Clean up everybody cle...,라바 키즈동요 Part3,0:02
...,...,...,...,...,...,...,...,...,...,...,...
4711,엄마가 화났다,인기작가 최숙희의 그림책. 아이들에게 건네는 화해와 위로의 메시지! 아이와 엄마 사...,-,,키즈,학습,키즈,0:08,인기작가 최숙희의 그림책. 아이들에게 건네는 화해와 위로의 메시지! 아이와 엄마 사...,엄마가 화났다,0:08
4712,핑크퐁 자동차 동화,Lil Reds First Fire(용감한 소방차 뚜뚜와 뿌뿌). 불이 난 곳이면 ...,핑크퐁,,키즈,학습,키즈,0:04,Lil Reds First Fire(용감한 소방차 뚜뚜와 뿌뿌). 불이 난 곳이면 ...,핑크퐁 자동차 동화,0:04
4713,간질간질,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,-,,키즈,학습,키즈,0:06,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,간질간질,0:06
4714,월간 아기상어,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,아기상어,,키즈,학습,키즈,0:01,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,월간 아기상어,0:01


In [66]:
def convert_time_format(time_str):
    try:
        # 시간 형식으로 파싱 시도
        time_obj = datetime.strptime(time_str, '%H:%M')
        # 분으로 변환
        return f"{time_obj.hour * 60 + time_obj.minute}분"
    except ValueError:
        # 파싱 에러 발생 시 원래 문자열 반환
        return time_str

In [72]:
vod['running_time'][1] = convert_time_format(vod['running_time'][1])


In [76]:
l = vod.shape[0]
for i in range(l):
    try:
        vod['running_time'][i] = convert_time_format(vod['running_time'][i])
    except:
        print(vod['running_time'][i], i)

nan 54
nan 67
nan 136
nan 166
nan 232
nan 477
nan 541
nan 566
nan 595
nan 659
nan 793
nan 1113
nan 1145
nan 1217
nan 1267
nan 1571
nan 3489
nan 3823
nan 3922
nan 4059
nan 4121
nan 4148
nan 4174
nan 4263
nan 4301
nan 4396
nan 4416


In [77]:
vod.fillna('', inplace=True)

In [78]:
vod

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY,vod_name,running_time
0,핑크퐁 동물 동요,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,,키즈,기타,키즈,0:01,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁 동물 동요,1분
1,라바 키즈동요 Part4,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...,"엘로우,레드",,키즈,기타,키즈,0:02,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...,라바 키즈동요 Part4,2분
2,지니강이 플러스 시즌7,친구들 안녕! 지니와 강이와 함께 신나는 장난감 놀이와 두근두근 대결놀이까지! 함께...,"헤이지니,럭키강이",,키즈,기타,키즈,0:11,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...,지니강이 플러스 시즌7,11분
3,엉뚱남매 요리조리쇼,먹을 수 있는 물병. 지구 환경을 지키자는 엉뚱남매. 하지만 말처럼 쉽게 지켜지지가...,"기원,예현,홍윤화",,키즈,기타,키즈,0:11,먹을 수 있는 물병. 지구 환경을 지키자는 엉뚱남매. 하지만 말처럼 쉽게 지켜지지가...,엉뚱남매 요리조리쇼,11분
4,라바 키즈동요 Part3,CLEAN UP SONG. Clean up Clean up everybody cle...,-,,키즈,기타,키즈,0:02,CLEAN UP SONG. Clean up Clean up everybody cle...,라바 키즈동요 Part3,2분
...,...,...,...,...,...,...,...,...,...,...,...
4711,엄마가 화났다,인기작가 최숙희의 그림책. 아이들에게 건네는 화해와 위로의 메시지! 아이와 엄마 사...,-,,키즈,학습,키즈,0:08,인기작가 최숙희의 그림책. 아이들에게 건네는 화해와 위로의 메시지! 아이와 엄마 사...,엄마가 화났다,8분
4712,핑크퐁 자동차 동화,Lil Reds First Fire(용감한 소방차 뚜뚜와 뿌뿌). 불이 난 곳이면 ...,핑크퐁,,키즈,학습,키즈,0:04,Lil Reds First Fire(용감한 소방차 뚜뚜와 뿌뿌). 불이 난 곳이면 ...,핑크퐁 자동차 동화,4분
4713,간질간질,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,-,,키즈,학습,키즈,0:06,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,간질간질,6분
4714,월간 아기상어,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,아기상어,,키즈,학습,키즈,0:01,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,월간 아기상어,1분


In [84]:
vod.to_csv('vod_list_addinfo.csv', encoding='utf-8')